In [1]:
import torch
import pyro
from pyro.optim import Adam, ClippedAdam
import congas as cg
from congas.models import LatentCategorical
from pyro.infer import TraceMeanField_ELBO,TraceEnum_ELBO, TraceGraph_ELBO



Bad key "text.kerning_factor" on line 4 in
/Users/salvatore/miniconda3/envs/hmobster/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import pickle

data_file = open("data_input_lucrezia.pkl",'rb')

data = pickle.loads(data_file.read())
data_file.close()

param_file =  open("pars_lucrezia.pkl",'rb')

param = pickle.loads(param_file.read())
param_file.close()


interface = cg.Interface()

In [3]:
pyro.enable_validation()

In [4]:
param["init_probs"] = 0.6
param["latent_type"] = "B"
param["lambda"] = 0.
param["K"] = 2
param["Temperature"] = 10

In [5]:
interface.set_model(LatentCategorical)
interface.set_optimizer(ClippedAdam)
interface.set_loss(TraceGraph_ELBO)
interface.initialize_model(data)
interface.set_model_params(param)

In [6]:
ll = interface.run(steps = 1000, param_optimizer = {"lr":0.005})



Running LatentCategorical on 3768 cells with 20 segments for 1000 steps and 2 modalities

Done!


ELBO: 1.229822477  : 100%|██████████| 1000/1000 [00:49<00:00, 20.07it/s]


In [7]:

import numpy as np

lr = interface.learned_parameters()
ICs = interface.calculate_ICs()


Computing assignment probabilities
Computing information criteria.


In [8]:
lr

{'mixture_weights_rna': array([0.5, 0.5], dtype=float32),
 'mixture_weights_atac': array([0.6333997, 0.3666003], dtype=float32),
 'NB_size_rna': array([149.99998, 149.99998, 149.99998, 149.99998, 149.99998, 149.99998,
        149.99998, 149.99998, 149.99998, 149.99998], dtype=float32),
 'segment_factor_rna': array([14.138147 ,  4.726283 ,  2.2323036,  4.872331 ,  3.2121391,
        12.321158 , 12.533496 ,  1.9288127,  3.9793284,  2.9039512],
       dtype=float32),
 'NB_size_atac': array([36.52858 , 14.793733, 10.956519, 15.497696, 11.785625, 17.42074 ,
        35.842995, 11.016844, 13.138355, 11.105012], dtype=float32),
 'segment_factor_atac': array([14.601969 , 12.444009 , 20.759323 ,  7.207906 ,  7.0514364,
        17.768087 , 20.184986 ,  8.464347 ,  7.9246697, 10.382282 ],
       dtype=float32),
 'CNV_probabilities': array([[[0.00304677, 0.00304677, 0.00304677, 0.00304677, 0.00304677,
          0.984766  ],
         [0.00304677, 0.00304677, 0.9847661 , 0.00304677, 0.00304677,
     

In [5]:
lk_atac = interface._model.likelihood(lr_t, "atac")

NameError: name 'interface' is not defined

In [43]:
from congas import log_sum_exp
lk_atac = lk_atac + torch.log(lr_t["mixture_weights_atac"]).reshape([2,1,1])

In [46]:
torch.exp(lk_atac - log_sum_exp(lk_atac))

tensor([[[9.9939e-01, 9.9989e-01, 1.4069e-04,  ..., 6.5166e-03,
          5.3351e-01, 1.5664e-07],
         [9.3591e-01, 1.6383e-01, 1.8467e-02,  ..., 6.9422e-01,
          8.2800e-01, 1.4531e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         ...,
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01]],

        [[6.1418e-04, 1.1402e-04, 9.9986e-01,  ..., 9.9348e-01,
          4.6649e-01, 1.0000e+00],
         [6.4089e-02, 8.3617e-01, 9.8153e-01,  ..., 3.0578e-01,
          1.7200e-01, 8.5469e-01],
         [4.9646e-01, 4.9646e-01, 4.9646e-01,  ..., 4.9646e-01,
          4.9646e-01, 4.9646e-01],
         ...,
         [4.9646e-01, 4.9646e-01, 4.9646e-01,  ..., 4.9646e-01,
          4.964

TypeError: exp(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

dict_keys(['mixture_weights_rna', 'mixture_weights_atac', 'NB_size_rna', 'segment_factor_rna', 'NB_size_atac', 'segment_factor_atac', 'CNV_probabilities', 'CNA', 'assignment_probs_rna', 'assignment_rna', 'assignment_probs_atac', 'assignment_atac'])